# shortCardiac

standardized, simplified and accelerated - cardiac evaluation of tomorrow

In [4]:
import json
import pickle
import time
from multiprocessing import Pool, freeze_support

import natsort
from tqdm import tqdm

import numpy as np
from xml.dom import minidom
import cv2
import nibabel as nib
import numpy as np
import pydicom
import csv
import glob
import os
import pandas as pd
import numpy as np

import moviepy.video.io.ImageSequenceClip
import pydicom
from PIL import Image
from natsort.natsort import natsorted

import math

import numpy as np
from numba import jit

import SimpleITK as sitk
import numpy as np
import pydicom
import radiomics
import six
from PIL import Image, ImageDraw
from radiomics import firstorder, glcm, glrlm, glszm, shape2D
from radiomics import setVerbosity


from scipy.ndimage.morphology import binary_fill_holes

ImportError: cannot import name 'firstorder' from 'radiomics' (c:\users\ludge\appdata\local\programs\python\python38\lib\site-packages\radiomics\__init__.py)

In [ ]:
# Default RunConfiguration


class RunConfiguration(object):
    def __init__(self):
        # Using the DebugMode, only one image is loaded; however, several outputs are generated,
        # which visualize the functioning of the script and the current calculations step by step.
        self.DEBUG = False

        self.coord_mode = 'nii'

        ##############################################
        # Activate and deactivate calculation
        # --------------------------------------------
        # More specific settings can change on the
        # end of the config file
        ##############################################
        self.calc_features = True
        # self.calc_right_ventricle_endocard = True
        # self.calc_left_ventricular_epicard = True
        # self.calc_left_ventricular_endocard = True

        # Used number of processes
        self.worker = 0

        ###############################################
        # Image Mode Selection
        ##############################################
        self.img_transparent = True
        self.save_pngs = True
        self.crop_img = True  # Boolean specifying whether the image should be cropped to the heart center
        self.crop_x = 0.25  # percent as float in range 0 - 1
        self.crop_y = 0.25  # percent as float in range 0 - 1
        # first image
        self.first_img_overlay_dicom = True
        self.first_img_overlay_rois = False
        self.first_img_overlay_rois_alpha = 0.4  # hint: float in range 0 - 1
        self.first_img_overlay_EI = True
        self.first_img_overlay_lines = False
        # second image
        self.second_img = True
        self.second_img_overlay_dicom = False
        self.second_img_overlay_rois = True
        self.second_img_overlay_rois_alpha = 0.2  # hint: float in range 0 - 1
        self.second_img_overlay_EI = False
        self.second_img_overlay_lines = True


        # ############################################## Resolution Settings
        # ############################################# Factor by which the read coordinates are scaled up for a more
        # precise calculation (integer between 1 and infinity). Note: The higher the resolution the longer the
        # calculation and the more RAM is needed. The script was optimized using factor 8
        self.resize_polygon_factor = 8

        ###############################################
        # Coordinates Correction Setting
        ###############################################
        # Indicates whether the heart angle is to be corrected in a standardized manner.
        self.angle_correction = True
        # Specifies whether the heart angle should be corrected in a standardized way.
        self.smooth_resizing = True

        ################################################
        # Calculations for the right ventricle endocard
        ###############################################
        # Indication of the name used by Circle for the contour of the right ventricle.
        self.rv_name_or_nr = "sarvendocardialContour"
        # Settings for measuring the ventral contour of the right ventricle
        self.rv_endo_ventral_angle_min = 0
        self.rv_endo_ventral_angle_max = 180
        self.rv_endo_ventral_angle_step_size = 15
        # Settings for measuring the dorsal contour of the right ventricle
        self.rv_endo_dorsal_angle_min = 0
        self.rv_endo_dorsal_angle_max = 180
        self.rv_endo_dorsal_angle_step_size = 15

        ################################################
        # Calculations for the left ventricular epicard
        ###############################################
        self.lv_epi_name_or_nr = "saepicardialContour"
        self.lv_epi_angle_min = 0
        self.lv_epi_angle_max = 360
        self.lv_epi_angle_step_size = 15

        ################################################
        # Calculations for the left ventricular endocard
        # ################################################
        self.lv_endo_name_or_nr = "saendocardialContour"
        self.lv_endo_angle_min = 0
        self.lv_endo_angle_max = 360
        self.lv_endo_angle_step_size = 15

    def generate_dict(self):
        return {key: value for key, value in self.__dict__.items() if not key.startswith('__') and not callable(key)}

    def save_to_json(self, save_file):
        information = self.generate_dict()
        js = json.dumps(information)

        with open(save_file if '.json' in save_file else save_file + '.json', 'w+') as f:
            f.write(js)

    def load_from_json(self, json_file):
        pass


In [ ]:
def keepElementNodes(nodes):
    """ Get the element nodes """
    nodes2 = []
    for node in nodes:
        if node.nodeType == node.ELEMENT_NODE:
            nodes2 += [node]
    return nodes2


def parseContours(node):
    """
        Parse a Contours object. Each Contours object may contain several contours.
        We first parse the contour name, then parse the points and pixel size.
        """
    contours = {}
    for child in keepElementNodes(node.childNodes):
        contour_name = child.getAttribute('Hash:key')
        sup = 1
        for child2 in keepElementNodes(child.childNodes):
            if child2.getAttribute('Hash:key') == 'Points':
                points = []
                for child3 in keepElementNodes(child2.childNodes):
                    x = float(child3.getElementsByTagName('Point:x')[0].firstChild.data) #+ 100
                    y = float(child3.getElementsByTagName('Point:y')[0].firstChild.data) #+ 100
                    points += [[x, y]]
            if child2.getAttribute('Hash:key') == 'SubpixelResolution':
                sub = int(child2.firstChild.data)
        points = np.array(points)
        points /= sub
        contours[contour_name] = points
    return contours


def traverseNode(node, uid_contours):
    """ Traverse the nodes """
    child = node.firstChild
    while child:
        if child.nodeType == child.ELEMENT_NODE:
            # This is where the information for each dicom file starts
            if child.getAttribute('Hash:key') == 'ImageStates':
                for child2 in keepElementNodes(child.childNodes):
                    # UID for the dicom file
                    uid = child2.getAttribute('Hash:key')
                    for child3 in keepElementNodes(child2.childNodes):
                        if child3.getAttribute('Hash:key') == 'Contours':
                            contours = parseContours(child3)
                            if contours:
                                uid_contours[uid] = contours
        traverseNode(child, uid_contours)
        child = child.nextSibling


def parseFile(xml_name, coord_file):
    """ Parse a cvi42 xml file """
    dom = minidom.parse(xml_name)
    uid_contours = {}
    traverseNode(dom, uid_contours)

    data = {}
    for uid, contours in uid_contours.items():
        data[uid] = contours
    with open(coord_file, 'wb') as f:
        pickle.dump(data, f)

In [ ]:
class CoordReader:
    def __init__(self, config):
        self.config = config

    def load_coordinates(self, file):
        """
        Import of the coordinates segmented in Circle. If the file is a pickle file ('*.pkl'), the coordinates are imported directly.
        Alternatively, the data is first prepared and then imported.
        """
        if '.pkl' not in file:
            print(
                'Invalid file format! Please use a preparation function to translate the coordinates to the correct format.')
            return None
        with open(file, 'rb') as f:
            return pickle.load(f)

    def preparation_cvi42(self, cvi42_file, save_file_name=None):
        """
        Reading out the coordinates and assigning them to the dicom files

                Parameters:
                        cvi42_file (str): circle-xml file with coordinates
                        uis (list): list with included dicom uis
                        coord_attributes = ....

                Returns:
                        coordinates (dict: dict with dicom ui as key
        """
        save_file_name = save_file_name if save_file_name is not None else cvi42_file + '.pkl'
        parseFile(cvi42_file, save_file_name)

    def preparation_nii(self, nii_file, dcm_sorted, save_file_name=None):
        ids = [self.config.rv_name_or_nr, self.config.lv_epi_name_or_nr, self.config.lv_endo_name_or_nr]
        mask = self.__load_nifti(nii_file)
        coords = self.__get_polygon_of_mask(mask, dcm_sorted, ids)

        with open(save_file_name, 'wb') as f:
            pickle.dump(coords, f)

    def __load_nifti(self, nii_file: str) -> np.ndarray:
        nimg = nib.load(nii_file)
        return nimg.get_fdata()[:, :, ::-1].transpose(1, 0, 2)

    def __get_polygon_of_mask(self, mask, dcm_sorted, ids, up_scaling=4):
        coords = {}

        for i, dcm_file in enumerate(dcm_sorted):
            ui = pydicom.dcmread(dcm_file).SOPInstanceUID
            coords[ui] = {}
            slice_mask = mask[:, :, i].copy().astype('int16')
            r, g, b = slice_mask.copy(), slice_mask.copy(), slice_mask.copy()

            r[r != int(ids[0])] = 0
            r[r == int(ids[0])] = 255
            g[g != int(ids[1])] = 0
            g[g == int(ids[1])] = 255
            b[b != int(ids[2])] = 0
            b[b == int(ids[2])] = 255
            temp_rgb = cv2.merge([r, g, b])
            temp_rgb = cv2.blur(temp_rgb, (2,2))
            temp_rgb = cv2.resize(temp_rgb, dsize=(temp_rgb.shape[0] * up_scaling, temp_rgb.shape[0] * up_scaling),
                                  interpolation=cv2.INTER_AREA)
            temp_rgb = cv2.blur(temp_rgb, (5, 5))
            slice_mask = np.zeros((temp_rgb.shape[0], temp_rgb.shape[1]))
            r, g, b = cv2.split(temp_rgb)
            for x in range(slice_mask.shape[0]):
                for y in range(slice_mask.shape[1]):
                    if all([r[x, y] < 100, g[x, y] < 100, b[x, y] < 100]):
                        continue
                    slice_mask[x, y] = np.argmax(np.array([r[x, y], g[x, y], b[x, y]])) + 1
            for ii, id in enumerate(ids):
                id = int(id)

                temp = slice_mask.copy()
                temp[temp != id] = 0
                temp[temp == id] = 1
                temp = binary_fill_holes(temp).astype(int)

                coords[ui][str(id)] = mask_to_polygon(temp) / up_scaling

        return coords


In [ ]:
def load_DICOMs(folder: str, fast_mode: bool = False) -> [list, list]:
    """
    Read out all Dicom files in the directory and all subdirectories.

         Parameters:
                folder (str): Folder
                fast_mode (bool): Specifies that all Dicom images have the suffix '*.dcm'; this allows much faster browsing of the directory tree.

        Returns:
            dcms = list of all found Dicom files
            uis = list with all imported dicom uis
    """
    print("Start loading of DICOM images")
    # load list with all dicom files
    dcms, uis = [], []
    for (dirpath, dirnames, filenames) in os.walk(folder):
        for filename in filenames:
            if fast_mode and os.path.splitext(filename)[-1] == 'dcm':
                continue
            dcm_file = os.path.join(dirpath, filename)
            # Checks if the file is a DICOM file; if so, the file and ui will saved
            if is_file_a_dicom(dcm_file):
                uis.append(pydicom.dcmread(dcm_file).SOPInstanceUID)
                dcms.append(dcm_file)
    return dcms, uis


def load_dcm_as_PIL(dcm_file: str, resize_factor: int =1):
    """
    Returns resized dicom image as Pillow RGB Image

            Parameters:
                    dcm_file (str): Path to dicom image
                    resize_factor (int): rescaling factor for DICOM images

            Returns:
                    dcm_img (Pillow Image): dcm_image as RGB image
    """

    dicom = pydicom.dcmread(dcm_file).pixel_array.astype('float32')

    # rescale DICOM image to RGB scaling
    dicom_rgb = dicom / dicom.max() * 255

    # resizing and converting to RGB
    return Image.fromarray(dicom_rgb).resize(
        (dicom.shape[1] * resize_factor,
         dicom.shape[0] * resize_factor)).convert('RGB')


def save(results, dicom_folder):
    customized_saving_technical_note(results, dicom_folder)


def customized_saving_technical_note(results, dicom_folder):
    """
    Customized function to save results, other formats could add
    """
    # D0 = septum length
    def calc_D0(params, params_name):
        index_0 = params_name.index("septum_center_to_right_ventricle_endo_ventral 0° [mm]")
        index_180 = params_name.index("septum_center_to_right_ventricle_endo_ventral 180° [mm]")
        D0 = params[index_0] + params[index_180]
        D0_name = "D0"
        return D0, D0_name

    def params_to_string(params):
        """
        convert params to string

        :param params: tuple of list with params

        :return: string with params
        """
        params = [str(p).replace(',', '\\').replace('.', ',') + ';' for p in params]
        return "".join(s for s in params)

    first_img = True
    for i, result in enumerate(results):
        file, params, params_name = result
        if len(params) == 0:
            continue
        D0, D0_name = calc_D0(params, params_name)
        if first_img:
            string_results = 'Mode;file;' + ''.join([name + ';' for name in params_name + [D0_name]]) + '\n'
            first_img = False
        df = pydicom.dcmread(file)
        Mode = file.replace(dicom_folder, '').split('\\')[1]
        string_results += Mode + ';' + file.replace(dicom_folder, '') + ';' + params_to_string(params + [D0]) + '\n'
    if first_img:
        print("Error")
        return None
    with open(dicom_folder + r'\results_de.csv', 'w+') as csv_file:
        csv_file.writelines(string_results)
    with open(dicom_folder + r'\results_eng.csv', 'w+') as csv_file:
        csv_file.writelines(string_results.replace(',', '.'))


def generate_mp4(folder: str, fps: int = 3) -> None:
    """
    creates an mp4-video with the saved images

    :param folder: path to saved images
    :param fps: Video fps
    """
    images = natsorted(glob.glob(folder + os.sep + "*.png"))
    clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(images, fps=fps)
    clip.write_videofile(folder + os.sep + 'result_video.mp4')


def load_result_csv(file: str) -> pd.DataFrame:
    """
    load result.csv file as pandas dataframe - worked with ',' and '.' separators

    :param file: path to csv-file
    :return: pandas dataframe
    """
    with open(file) as file_obj:
        reader_obj = csv.reader(file_obj, delimiter=";")
        header = None
        lines = []

        def transform_value(value):
            try:
                value = float(value.replace(',', '.'))
            except ValueError:
                pass
            return value

        for i, row in enumerate(reader_obj):
            if i == 0:
                header = row
                continue

            lines.append([transform_value(_.replace(',', '.')) for _ in row])
    if header is None:
        return None
    return pd.DataFrame(np.array(lines, dtype=object), columns=header)


In [ ]:
def calc_mask_of_polygon(dcm_img: str, coords: dict, name: str, resize_polygon_factor: int) -> np.ndarray:
    """

    :param dcm_img: path to DICOM image
    :param coords: dict with all segmentation coordinates
    :param name: name of segmentation - key for coords dict
    :param resize_polygon_factor: factor between segmentation coords and original DICOM image

    :return: bool mask with ones and zeros
    """
    dcm_shape = pydicom.dcmread(dcm_img).pixel_array.shape
    img = Image.new('L', (dcm_shape[1], dcm_shape[0]), 0)
    img_draw = ImageDraw.Draw(img)
    c = tuple([(c1 / resize_polygon_factor, c2 / resize_polygon_factor) for c1, c2 in list(coords.get(name))])
    img_draw.polygon(c, outline=1, fill=1)
    return np.array(img)


def calc_radiomics(dcm_file: str, mask: np.ndarray, name: str, normalize: bool = False, cf: dict = None):
    """
    function for calculation of radiomcs features - wrapper fuction for pyradiomics

    more information: https://pyradiomics.readthedocs.io/en/latest/
    """

    if cf is None:
        cf = {
            "shape_feature": True,
            "firstorder_feature": True,
            "glcm_feature": True,
            "glrlm_feature": True,
            "glszm_feature": True,
        }
    setVerbosity(60)
    dcm_sitk = sitk.GetImageFromArray(pydicom.dcmread(dcm_file).pixel_array)
    if normalize:
        dcm_sitk = radiomics.imageoperations.normalizeImage(dcm_sitk)
    mask_sitk = sitk.GetImageFromArray(mask)

    def extract_results(res, feauture_class=None):
        feature, feature_names = [], []
        for key, val in six.iteritems(res):
            if 'Version' in key or 'Settings' in key or 'Configuration' in key:
                continue
            feature.append(val)
            _ = "" if feauture_class is None else feauture_class + "_"
            feature_names.append(_ + name + '_' + key.replace('original_', '').replace('diagnostics_', ''))

        return feature, feature_names

    feature_list = []
    feature_names_list = []

    if cf["shape_feature"]:
        shape_extractor = shape2D.RadiomicsShape2D(dcm_sitk, mask_sitk)
        shape_results = shape_extractor.execute()
        feature, feature_names = extract_results(shape_results, "shape")
        feature_list.append(feature)
        feature_names_list.append(feature_names)

    if cf["firstorder_feature"]:
        firstorder_extractor = firstorder.RadiomicsFirstOrder(dcm_sitk, mask_sitk)
        firstorder_results = firstorder_extractor.execute()
        feature, feature_names = extract_results(firstorder_results, "firstorder")
        feature_list.append(feature)
        feature_names_list.append(feature_names)

    if cf["glcm_feature"]:
        gclm_extractor = glcm.RadiomicsGLCM(dcm_sitk, mask_sitk)
        glcm_results = gclm_extractor.execute()
        feature, feature_names = extract_results(glcm_results, "glcm")
        feature_list.append(feature)
        feature_names_list.append(feature_names)

    if cf["glrlm_feature"]:
        glrlm_extractor = glrlm.RadiomicsGLRLM(dcm_sitk, mask_sitk)
        glrlm_results = glrlm_extractor.execute()
        feature, feature_names = extract_results(glrlm_results, "glrlm")
        feature_list.append(feature)
        feature_names_list.append(feature_names)

    if cf["glszm_feature"]:
        glszm_extractor = glszm.RadiomicsGLSZM(dcm_sitk, mask_sitk)
        glszm_results = glszm_extractor.execute()
        feature, feature_names = extract_results(glszm_results, "glszm")
        feature_list.append(feature)
        feature_names_list.append(feature_names)

    return flatted_list(feature_list), flatted_list(feature_names_list)

In [ ]:
from abc import ABC

import matplotlib.pyplot as plt

from shortCardiacBackend.SIConvertion import convert_params
from shortCardiacBackend.loadAndSave import load_dcm_as_PIL
from shortCardiacBackend.supportFunction import *
from shortCardiacBackend.transformContours import *
from shortCardiacBackend.transformPointsAndVectors import *
from shortCardiacBackend.visualisation import *
from shortCardiacBackend.radiomics import calc_radiomics, calc_mask_of_polygon
from shortCardiacBackend.ShowCalculationsStepByStep import *


def display(image):
    plt.imshow(image)
    plt.show()


def check_coords(coords, config):
    if config.rv_name_or_nr not in coords.keys():
        return False
    if config.lv_endo_name_or_nr not in coords.keys():
        return False
    if config.lv_epi_name_or_nr not in coords.keys():
        return False
    return True


class ShortCardiac(ABC):
    def __init__(self, config, dcm_file, coords):
        self.config = config
        self.dcm_file = dcm_file

        self.calculable = check_coords(coords, config)
        # show_segmentation(config, load_dcm_as_PIL(dcm_file, 1), coords)

        # Read in the dcm_image as a PILLOW image for the visualization of the calculations.
        self.dcm_img = load_dcm_as_PIL(dcm_file, config.resize_polygon_factor)
        coords_resizing(coords, config.resize_polygon_factor, config.smooth_resizing)
        self.showCalculationStepByStep = False

        if self.calculable:

            find_ref_points(coords, config)
            if coords["sacardialRefPoint"] is None:
                self.calculable = False

        if not self.calculable:
            self.dcm_img = show_segmentation(config, load_dcm_as_PIL(dcm_file, config.resize_polygon_factor), coords)
            if self.config.save_pngs:
                self.dcm_img.save(self.dcm_file.replace('.dcm', '.png'), transparent=self.config.img_transparent)

        if self.calculable:
            if self.showCalculationStepByStep:
                show_Ref_points_and_septum_axis(self.dcm_img, coords, self.dcm_file)
            if self.showCalculationStepByStep:
                c_ = deepcopy(coords)
            correct_segmentations(coords, config)
            if self.showCalculationStepByStep:
                show_coord_correction(self.dcm_img, coords, c_, config)

            # Resizing the circle coordinates by the factor {resize_polygon_factor} allows a more stable calculation.
            # In addition, the missing points are added and the ROIs spline interpolated (smoothed).
            # For more information see function: smooth_coord_resizing and the documentation scipy.interpolate.splprep
            # (https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.splprep.html).
            # show_segmentation(config, load_dcm_as_PIL(dcm_file, 1), coords)

            # Division of the coordinates of the right ventricle into a ventral and dorsal contour.
            # The calculation uses the SAX Reference points if available:
            # --> S0 = superior SAX Reference point [sacardialRefPoint]
            # --> S1 = inferior SAX Reference point [sacardialInferiorRefPoint]
            # if these are not present they are automatically calculated based on the contours of the right ventricle and the
            # .... calculated

            # Detection of reference points: sacardialRefPoint and sacardialInferiorRefPoint if they are not present in the dataset.

            # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! STILL NEEDS TO BE IMPLEMENTED !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

            # Calculation of septum alignment and correction for standardized measurement
            septum_angle = calc_cardiac_angle(coords)
            #if abs(septum_angle) > 7:
            #    self.showCalculationStepByStep = True
            #else:
            #    self.showCalculationStepByStep = False
            center_img = (self.dcm_img.size[0] / 2, self.dcm_img.size[1] / 2)
            if self.config.DEBUG:
                print(
                    f"ORGINAL DICOM IMAGE (left) AND DICOM IMAGE (right) AFTER THE HEART AXIS HAS BEEN CORRECTED BY {septum_angle}°.")
                img_conc = concatenate_images(self.dcm_img, rotate_img(self.dcm_img, - septum_angle))
                display(img_conc)
                print(" \n \n")
            if self.showCalculationStepByStep:
                coords_pre = deepcopy(coords)
            if config.angle_correction:
                for key in coords.keys():
                    coords[key] = np.array(rotate_points(coords[key], center_img, septum_angle))
            if self.showCalculationStepByStep:
                show_angle(self.dcm_img, coords_pre, rotate_img(self.dcm_img, - septum_angle), coords, center_img,
                           septum_angle)
            self.septum_angle = septum_angle
            self.coords = coords

        self.septum_angle_name = "Septum_angle [°]"

        #    self.__init_parameters()

        # def __init_important_parameters(self):
        # quantitative_septum_evaluation
        self.septum_axis = [None]
        self.septum_axis_name = ["SeptumRefPoint", "SeptumInferiorRefPoint"]
        self.septum_center = None
        self.septum_center_name = "Center point between RefPoints"

        # quantitative_right_ventricle_evaluation
        self.Center_rv_endo = None
        self.Center_rv_endo_name = "Center_rv_endo"
        self.Area_rv_endo = None
        self.Area_rv_endo_name = "Area_rv_endo [cm^2]"
        self.Area_between_rv_endo_and_septum_axis = None
        self.Area_between_rv_endo_and_septum_axis_name = "Area_between_rv_endo_and_septum_axis [cm^2]"
        self.distance_rv_endo_ventral_to_septum_center = None
        self.distance_rv_endo_ventral_to_septum_center_name = [
            f"septum_center_to_right_ventricle_endo_ventral {i}° [mm]" for i in
            range(self.config.rv_endo_ventral_angle_min,
                  self.config.rv_endo_ventral_angle_max + 1,
                  self.config.rv_endo_ventral_angle_step_size)]
        self.distance_rv_endo_dorsal_to_septum_center = None
        self.distance_rv_endo_dorsal_to_septum_center_names = [f"septum_center_to_rv_endo_dorsal {i}° [mm]"
                                                               for i in
                                                               range(self.config.rv_endo_dorsal_angle_min,
                                                                     self.config.rv_endo_dorsal_angle_max + 1,
                                                                     self.config.rv_endo_dorsal_angle_step_size)]
        self.EI_line_rv_endo_0 = None
        self.EI_line_rv_endo_0_name = "EI_line_rv_endo_0 [mm]"
        self.EI_line_rv_endo_90 = None
        self.EI_line_rv_endo_90_name = "EI_line_rv_endo_90 [mm]"
        self.Septum_axis_to_rv_endo = None
        self.Septum_axis_to_rv_endo_name = "Septum_axis_to_rv_endo [mm]"
        self.Line_Intersection_EI_rv_endo = None
        self.Line_Intersection_EI_rv_endo_name = "Line_Intersection_EI_rv_endo"
        self.scope_rv_endo = None
        self.scope_rv_endo_name = "scope_rv_endo [mm]"

        # quantitative_saepicardial_contour_evaluation
        self.Center_lv_epi = None
        self.Center_lv_epi_name = "Center_lv_epi"
        self.Area_lv_epi = None
        self.Area_lv_epi_name = "Area_lv_epi [mm^2]"
        self.distance_lv_epi = None
        self.distance_lv_epi_name = [f"Distance_lv_epi {i}° [mm]" for i in
                                     range(self.config.lv_epi_angle_min,
                                           self.config.lv_epi_angle_max,
                                           self.config.lv_epi_angle_step_size)]
        self.EI_line_lv_epi_0 = None
        self.EI_line_lv_epi_0_name = "EI_line_lv_epi_0 [mm]"
        self.EI_line_lv_epi_90 = None
        self.EI_line_lv_epi_90_name = "EI_line_lv_epi_90 [mm]"
        self.Line_intersection_EI_lv_epi = None
        self.Line_Intersection_EI_lv_epi_name = "Line_Intersection_EI_lv_epi"
        self.scope_lv_epi = None
        self.scope_lv_epi_name = "scope_lv_epi [mm]"

        # quantitative_saendocardial_contour_evaluation
        self.Center_lv_endo = None
        self.Center_lv_endo_name = "Center_lv_endo"
        self.Area_lv_endo = None
        self.Area_lv_endo_name = "Area_lv_endo [mm^2]"
        self.EI_line_lv_endo_0 = None
        self.EI_line_lv_endo_0_name = "EI_line_lv_endo_0 [mm]"
        self.EI_line_lv_endo_90 = None
        self.EI_line_lv_endo_90_name = "EI_line_lv_endo_90 [mm]"
        self.Line_Intersection_EI_lv_endo = None
        self.Line_Intersection_EI_lv_endo_name = "Line_Intersection_EI_lv_endo"
        self.distance_lv_endo = None
        self.distance_lv_endo_name = [f"Distance_lv_endo {i}° [mm]" for i in
                                      range(self.config.lv_endo_angle_min,
                                            self.config.lv_endo_angle_max,
                                            self.config.lv_endo_angle_step_size)]
        self.scope_lv_endo = None
        self.scope_lv_endo_name = "scope_lv_endo [mm]"

        self.merged_params = None
        self.merged_param_names = None

    def run(self):
        if not self.calculable:
            self.merge_results()
            return self.dcm_file, [], self.merged_param_names
        try:
            self.feature_extraction()
            self.quantitative_septum_evaluation()
            self.quantitative_right_ventricle_evaluation()
            self.quantitative_saendocardial_contour_evaluation()
            self.quantitative_saepicardial_contour_evaluation()
            self.merge_results()
        except:
            self.merge_results()
            return self.dcm_file, [], self.merged_param_names
        return self.get_results()

    def generate_mask(self):
        right_ventricel_mask = calc_mask_of_polygon(self.dcm_file, self.coords,
                                                    self.config.rv_name_or_nr,
                                                    self.config.resize_polygon_factor)
        saendocardialContour_mask = calc_mask_of_polygon(self.dcm_file, self.coords,
                                                         self.config.lv_endo_name_or_nr,
                                                         self.config.resize_polygon_factor)
        saepicardialContour_mask = calc_mask_of_polygon(self.dcm_file, self.coords,
                                                        self.config.lv_epi_name_or_nr,
                                                        self.config.resize_polygon_factor) - saendocardialContour_mask
        return right_ventricel_mask, saendocardialContour_mask, saepicardialContour_mask

    def feature_extraction(self):
        right_ventricel_mask, saendocardialContour_mask, saepicardialContour_mask = self.generate_mask()
        right_ventricel_feature = calc_radiomics(self.dcm_file, right_ventricel_mask,
                                                 name=self.config.rv_name_or_nr, normalize=True)
        saendocardialContour_feature = calc_radiomics(self.dcm_file, saendocardialContour_mask,
                                                      name=self.config.lv_endo_name_or_nr, normalize=True)
        saepicardialContour_feature = calc_radiomics(self.dcm_file, saepicardialContour_mask,
                                                     name=self.config.lv_epi_name_or_nr, normalize=True)
        self.radiomc_feature = {
            'right_ventricel': right_ventricel_feature,
            'saendocardialContour': saendocardialContour_feature,
            'saepicardialContour_feature': saepicardialContour_feature
        }

    def get_results(self):
        if self.merged_params is None:
            print("No calculation performed so far please use one of the following calculations first:"
                  " [quantitative_septum_evaluation, quantitative_right_ventricle_evaluation, "
                  "quantitative_saepicardial_contour_evaluation, quantitative_saendocardial_contour_evaluation]")
            return None
        else:
            return self.dcm_file, self.merged_params, self.merged_param_names

    def quantitative_septum_evaluation(self):
        """ ---------------------------------------------------------------------------------------------------------"""
        """ Calculation of septum alignment and septum center (midpoint between the two reference points S0 & S1) """
        """ ---------------------------------------------------------------------------------------------------------"""

        # The septum alignment is determined by the two reference points "sacardialRefPoint" and "sacardialInferiorRefPoint."
        # The points are determined automatically by Circle; should the points not exist in the loaded workspace,
        # then, they were filled in the previous step, "Detection of reference points."
        self.septum_axis = [self.coords["sacardialRefPoint"][0], self.coords["sacardialInferiorRefPoint"][0]]
        # Determination of the midpoint between the two reference points
        self.septum_center = calc_center_of_2_points(self.septum_axis[0], self.septum_axis[1])

        return (self.septum_axis, self.septum_axis_name), (self.septum_center, self.septum_center_name)

    def quantitative_right_ventricle_evaluation(self):
        """ ---------------------------------------------------------------------------------------------------------"""
        """ Calculations for the Right Ventricle """
        """ ---------------------------------------------------------------------------------------------------------"""

        # Calculation of the center of gravity \ center point for the "sarvendocardialContour" contour
        self.Center_rv_endo = calc_center_of_polygon(self.coords[self.config.rv_name_or_nr])
        # Calculation of the area for the contour "sarvendocardialContour" - Note: Unit here still Pixel^2 Conversion to cm^2 only in function "convert_params
        self.Area_rv_endo = calc_area(self.coords[self.config.rv_name_or_nr])

        # Calculation of the area spanned by the dorsal contour of the right atrium - Note Unit here still Pixel^2 Conversion to cm^2 only in function "convert_params".
        self.Area_between_rv_endo_and_septum_axis = calc_area(self.coords["sarvendocardialContourDorsalClosed"])

        # Calculation of Distance_Feauture for the ventral contour of the right ventricle.

        if self.septum_center is None:
            _ = self.quantitative_septum_evaluation()
        self.distance_rv_endo_ventral_to_septum_center = calc_outline(self.septum_center,
                                                                      self.coords["sarvendocardialContourVentral"],
                                                                      [i for i in range(self.config.rv_endo_ventral_angle_min,
                                                                                        self.config.rv_endo_ventral_angle_max + 1,
                                                                                        self.config.rv_endo_ventral_angle_step_size)])

        # Calculation of the Distance_Feauture for the dorsal contour of the right ventricle.
        self.distance_rv_endo_dorsal_to_septum_center = calc_outline(self.septum_center,
                                                                     self.coords["sarvendocardialContourDorsal"],
                                                                     [i for i in range(self.config.rv_endo_dorsal_angle_min,
                                                                                       self.config.rv_endo_dorsal_angle_max + 1,
                                                                                       self.config.rv_endo_dorsal_angle_step_size)])

        # Calculation of the longest diameters of the sarvendocardialContour at an angle of 0 and 90 degrees to the
        # cardiac axis for the calculation of the EI.
        self.EI_line_rv_endo_0 = calc_line_for_EI(self.coords[self.config.rv_name_or_nr], 0)

        self.EI_line_rv_endo_90 = calc_line_for_EI(self.coords[self.config.rv_name_or_nr], 90)

        # Calculation of the longest stretch antiparallel to the cardiac axis to describe the curvature of the right ventricle.
        self.Septum_axis_to_rv_endo = calc_line_for_EI(self.coords["sarvendocardialContourDorsalClosed"], 90)

        # Calculation of the intersection of L1 & L2
        self.Line_Intersection_EI_rv_endo = line_intersection(self.EI_line_rv_endo_0, self.EI_line_rv_endo_90)

        # Calculation of the scope for sarvendocardialContour
        self.scope_rv_endo = calc_polygon_length(self.coords[self.config.rv_name_or_nr])

        return {
            "Center_rv_endo": self.Center_rv_endo,
            "Center_rv_endo_name": self.Center_rv_endo_name,
            "Area_rv_endo": self.Area_rv_endo,
            "Area_rv_endo_name": self.Area_rv_endo_name,
            "Area_between_rv_endo_and_septum_axis": self.Area_between_rv_endo_and_septum_axis,
            "Area_between_rv_endo_and_septum_axis_name": self.Area_between_rv_endo_and_septum_axis_name,
            "distance_rv_endo_ventral_to_septum_center": self.distance_rv_endo_ventral_to_septum_center,
            "distance_rv_endo_ventral_to_septum_center_name": self.distance_rv_endo_ventral_to_septum_center_name,
            "distance_rv_endo_dorsal_to_septum_center": self.distance_rv_endo_dorsal_to_septum_center,
            "distance_rv_endo_dorsal_to_septum_center_names": self.distance_rv_endo_dorsal_to_septum_center_names,
            "EI_line_rv_endo_0": self.EI_line_rv_endo_0,
            "EI_line_rv_endo_0_name": self.EI_line_rv_endo_0_name,
            "EI_line_rv_endo_90": self.EI_line_rv_endo_90,
            "EI_line_rv_endo_90_name": self.EI_line_rv_endo_90_name,
            "Septum_axis_to_rv_endo": self.Septum_axis_to_rv_endo,
            "Septum_axis_to_rv_endo_name": self.Septum_axis_to_rv_endo_name,
            "Line_Intersection_EI_rv_endo": self.Line_Intersection_EI_rv_endo,
            "Line_Intersection_EI_rv_endo_name": self.Line_Intersection_EI_rv_endo_name,
            "scope_rv_endo": self.scope_rv_endo,
            "scope_rv_endo_name": self.scope_rv_endo_name,
        }

    def quantitative_saepicardial_contour_evaluation(self):
        """ ---------------------------------------------------------------------------------------------------------"""
        """ Calculations for saepicardialContour """
        """ ---------------------------------------------------------------------------------------------------------"""

        # Calculation of the center of gravity \ center point for the "saepicardialContour" contour
        self.Center_lv_epi = calc_center_of_polygon(self.coords[self.config.lv_epi_name_or_nr])

        # Calculation of the area for the contour "saepicardialContour" - Note: Unit here still Pixel^2 Conversion to cm^2 only in function "convert_params"
        self.Area_lv_epi = calc_area(self.coords[self.config.lv_epi_name_or_nr])

        # Calculation of the Distance_Feauture for the contour of the saepicardialContour
        self.distance_lv_epi = calc_outline(self.Center_lv_epi,
                                            self.coords[self.config.lv_epi_name_or_nr],
                                            [i for i in
                                             range(self.config.lv_epi_angle_min,
                                                   self.config.lv_epi_angle_max,
                                                   self.config.lv_epi_angle_step_size)])

        # Calculation of the longest diameters of the saepicardialContour at an angle of 0 and 90 degrees to the cardiac axis for the calculation of the EI.
        self.EI_line_lv_epi_0 = calc_line_for_EI(self.coords[self.config.lv_epi_name_or_nr], 0)
        self.EI_line_lv_epi_90 = calc_line_for_EI(self.coords[self.config.lv_epi_name_or_nr], 90)

        # Calculation of the intersection of L3 & L4
        self.Line_intersection_EI_lv_epi = line_intersection(self.EI_line_lv_epi_0, self.EI_line_lv_epi_90)

        # Calculation of the scope for sarvendocardialContour
        self.scope_lv_epi = calc_polygon_length(self.coords[self.config.lv_epi_name_or_nr])

        return {
            'Center_lv_epi': self.Center_lv_epi,
            'Center_lv_epi_name': self.Center_lv_epi_name,
            'Area_lv_epi': self.Area_lv_epi,
            'Area_lv_epi_name': self.Area_lv_epi_name,
            'distance_lv_epi': self.distance_lv_epi,
            'distance_lv_epi_name': self.distance_lv_epi_name,
            'EI_line_lv_epi_0': self.EI_line_lv_epi_0,
            'EI_line_lv_epi_0_name': self.EI_line_lv_epi_0_name,
            'EI_line_lv_epi_90': self.EI_line_lv_epi_90,
            'EI_line_lv_epi_90_name': self.EI_line_lv_epi_90_name,
            'Line_intersection_EI_lv_epi': self.Line_intersection_EI_lv_epi,
            'Line_Intersection_EI_lv_epi_name': self.Line_Intersection_EI_lv_epi_name,
            'scope_lv_epi': self.scope_lv_epi,
            'scope_lv_epi_name': self.scope_lv_epi_name}

    def quantitative_saendocardial_contour_evaluation(self):
        """--------------------------------------------------------------------------------------------------------- """
        """ Calculations for saendocardialContour """
        """ ---------------------------------------------------------------------------------------------------------"""

        # Calculation of the center of gravity \ center point for the "saendocardialContour" contour
        self.Center_lv_endo = calc_center_of_polygon(self.coords[self.config.lv_endo_name_or_nr])
        # Calculation of the area for the contour "saendocardialContour"
        # Note: Unit here still Pixel^2 Conversion to cm^2 only in function "convert_params
        self.Area_lv_endo = calc_area(self.coords[self.config.lv_endo_name_or_nr])
        # Calculation of the longest diameters of the saendocardialContour at an angle of 0 and 90 degrees to the cardiac axis for the analysis of the EI.
        self.EI_line_lv_endo_0 = calc_line_for_EI(self.coords[self.config.lv_endo_name_or_nr], 0)
        self.EI_line_lv_endo_90 = calc_line_for_EI(self.coords[self.config.lv_endo_name_or_nr], 90)
        # Calculation of the intersection of L5 & L6
        self.Line_Intersection_EI_lv_endo = line_intersection(self.EI_line_lv_endo_0, self.EI_line_lv_endo_90)

        # Calculation of the Distance_Feauture for the contour of the saendocardialContour
        self.distance_lv_endo = calc_outline(self.Center_lv_endo,
                                             self.coords[self.config.lv_endo_name_or_nr],
                                             [i for i in range(self.config.lv_endo_angle_min,
                                                               self.config.lv_endo_angle_max,
                                                               self.config.lv_endo_angle_step_size)])

        # Calculation of the scope for saendocardialContour
        self.scope_lv_endo = calc_polygon_length(self.coords[self.config.lv_endo_name_or_nr])

        return {
            'Center_lv_endo': self.Center_lv_endo,
            'Center_lv_endo_name': self.Center_lv_endo_name,
            'Area_lv_endo': self.Area_lv_endo,
            'Area_lv_endo_name': self.Area_lv_endo_name,
            'EI_line_lv_endo_0': self.EI_line_lv_endo_0,
            'EI_line_lv_endo_0_name': self.EI_line_lv_endo_0_name,
            'EI_line_lv_endo_90': self.EI_line_lv_endo_90,
            'EI_line_lv_endo_90_name': self.EI_line_lv_endo_90_name,
            'Line_Intersection_EI_lv_endo': self.Line_Intersection_EI_lv_endo,
            'Line_Intersection_EI_lv_endo_name': self.Line_Intersection_EI_lv_endo_name,
            'distance_lv_endo': self.distance_lv_endo,
            'distance_lv_endo_name': self.distance_lv_endo_name,
            'scope_lv_endo': self.scope_lv_endo,
            'scope_lv_endo_name': self.scope_lv_endo_name}

    def merge_results(self):
        """ ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------"""
        """ Merge the parameters into the line_params, poit_params and area_params """
        """ ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------"""
        line_params_names = [self.distance_rv_endo_ventral_to_septum_center_name] + [
            self.distance_rv_endo_dorsal_to_septum_center_names] + [
                                self.distance_lv_epi_name] + [self.distance_lv_endo_name] + [
                                [self.EI_line_rv_endo_0_name] + [self.EI_line_rv_endo_90_name]] + [
                                [self.EI_line_lv_epi_0_name] + [self.EI_line_lv_epi_90_name]] + [
                                [self.EI_line_lv_endo_0_name] + [self.EI_line_lv_endo_90_name]] + [
                                [self.Septum_axis_to_rv_endo_name]]
        line_params = [self.distance_rv_endo_ventral_to_septum_center] + [
            self.distance_rv_endo_dorsal_to_septum_center] + [
                          self.distance_lv_epi] + [
                          self.distance_lv_endo] + [[self.EI_line_rv_endo_0] + [self.EI_line_rv_endo_90]] + [
                          [self.EI_line_lv_epi_0] + [self.EI_line_lv_epi_90]] + [
                          [self.EI_line_lv_endo_0] + [self.EI_line_lv_endo_90]] + [[self.Septum_axis_to_rv_endo]]

        point_params = [self.Center_rv_endo] + list(self.septum_axis) + [self.Center_lv_epi] + [self.Center_lv_endo] + [
            self.septum_center] + [self.Line_Intersection_EI_rv_endo,
                                   self.Line_intersection_EI_lv_epi,
                                   self.Line_Intersection_EI_lv_endo]
        point_params_names = [self.Center_rv_endo_name] + self.septum_axis_name + [self.Center_lv_epi_name] + [
            self.Center_lv_endo_name] + [
                                 self.septum_center_name] + [self.Line_Intersection_EI_rv_endo_name,
                                                             self.Line_Intersection_EI_lv_epi_name,
                                                             self.Line_Intersection_EI_lv_endo_name]
        scope_params = [self.scope_rv_endo, self.scope_lv_epi, self.scope_lv_endo]
        scope_params_name = [self.scope_rv_endo_name, self.scope_lv_epi_name, self.scope_lv_endo_name]

        area_params = [self.Area_rv_endo, self.Area_between_rv_endo_and_septum_axis, self.Area_lv_epi,
                       self.Area_lv_endo]
        area_params_name = [self.Area_rv_endo_name, self.Area_between_rv_endo_and_septum_axis_name,
                            self.Area_lv_epi_name, self.Area_lv_endo_name]

        """ ---------------------------------------------------------------------------------------------------------"""
        """ Visualization of the calculated parameters """
        """ ---------------------------------------------------------------------------------------------------------"""

        if self.showCalculationStepByStep:
            show_center_points(self.dcm_img.rotate(-self.septum_angle), point_params)
            show_length_measurements(self.config, self.dcm_img, self.coords, line_params, point_params,
                                     angle=self.septum_angle,
                                     crop_img=self.config.crop_img if not self.config.DEBUG else False,
                                     center=self.septum_center)
            show_EIs_measurements(self.config, self.dcm_img, self.coords, line_params, point_params,
                                  angle=self.septum_angle,
                                  crop_img=self.config.crop_img if not self.config.DEBUG else False,
                                  center=self.septum_center)

        if (self.config.save_pngs or self.config.DEBUG) and self.calculable:
            self.dcm_img = visualization(self.config, self.dcm_img, self.coords, line_params, point_params,
                                         angle=self.septum_angle,
                                         crop_img=self.config.crop_img if not self.config.DEBUG else False,
                                         center=self.septum_center)
            if self.config.save_pngs:
                self.dcm_img.save(self.dcm_file.replace('.dcm', '.png'), transparent=self.config.img_transparent)
            else:
                display(self.dcm_img)

        if self.calculable:
            """ ---------------------------------------------------------------------------------------------------------"""
            """ Conversion of the determined parameters into SI units """
            """ ---------------------------------------------------------------------------------------------------------"""
            pixel_spacing = get_pixel_spacing(self.dcm_file)
            line_params = flatted_list(line_params)

            params = convert_params(line_params, point_params, area_params, scope_params,
                                    self.config.resize_polygon_factor,
                                    pixel_spacing)

            """ ---------------------------------------------------------------------------------------------------------"""
            """ Calculation of EIs """
            """ ---------------------------------------------------------------------------------------------------------"""

            EIs = [calc_EI(self.EI_line_rv_endo_0, self.EI_line_rv_endo_90),
                   calc_EI(self.EI_line_lv_epi_0, self.EI_line_lv_epi_90),
                   calc_EI(self.EI_line_lv_endo_0, self.EI_line_lv_endo_90)]

        EIs_name = ["EI_rv_endo", "EI_lv_epi", "EI_lv_endo"]

        """ ---------------------------------------------------------------------------------------------------------"""
        """ Reading Dicom tags and writing Dicomtags. """
        """ ---------------------------------------------------------------------------------------------------------"""

        # params_dicom_tags, params_dicom_tags_name = read_dicom_tags(self.dcm_file)
        # add_dicom_tags() Muss noch implementiert werden

        self.dcm_file = self.dcm_file
        if self.calculable:
            self.merged_params = [self.septum_angle] + flatted_list(params) + EIs + \
                                 self.radiomc_feature['right_ventricel'][0] + \
                                 self.radiomc_feature['saendocardialContour'][0] + \
                                 self.radiomc_feature['saepicardialContour_feature'][0]
        else:
            self.merged_params = None
        # + params_dicom_tags
        self.merged_param_names = [self.septum_angle_name] + flatted_list(
            line_params_names) + point_params_names + area_params_name + scope_params_name \
                                  + EIs_name
        if self.config.calc_features and self.calculable:
            self.merged_param_names += self.radiomc_feature['right_ventricel'][1] + \
                                       self.radiomc_feature['saendocardialContour'][1] + \
                                       self.radiomc_feature['saepicardialContour_feature'][
                                           1]  # + params_dicom_tags_name


def read_dicom_tags(file):
    """
    Reading out parameters stored in DicomTags

    :param file: string with path to dicom image
    :return:
    """
    ds = pydicom.dcmread(file)
    Trigger_time = ds[0x0018, 0x1060].value
    InstanceNumber = ds[0x0019, 0x0001].value
    # custom, custom_name = [], []
    # for custom_key in config.custom_keys:
    #    Atemfluss = ds[0x0019, 0x0002].value
    #    Atemvolumen = ds[0x0019, 0x0003].value
    # return [Trigger_time, InstanceNumber, Atemfluss, Atemvolumen], \
    #       ["Trigger_time", "InstanceNumber", "Atemfluss", "Atemvolumen"]
    return [Trigger_time, InstanceNumber], ["Trigger_time", "InstanceNumber"]


def calc_EI(d1, d2):
    d1 = length(get_vector(np.array(d1[0]), np.array(d1[1])))
    d2 = length(get_vector(np.array(d2[0]), np.array(d2[1])))
    return d1 / d2


def calc_cardiac_angle(coords):
    """
    Determination of the heart angle using the two reference points, sacardialRefPoint, and sacardialInferiorRefPoint.

            Parameters:
                    coords (dict): .....

            Returns:
                    angle (float): heart angle
    """
    sacardialRefPoint, sacardialInferiorRefPoint = coords.get("sacardialRefPoint")[0], \
                                                   coords.get("sacardialInferiorRefPoint")[0]
    angle = calc_angle(get_vector(sacardialRefPoint, sacardialInferiorRefPoint), np.array([0, -1]))
    if sacardialRefPoint[0] > sacardialInferiorRefPoint[0]:
        return - angle
    return angle


def calc_outline(origin, roi, angles):
    D = find_distance_with_angles_and_fixed_point(roi, angles, origin)
    return tuple([(D[key][0], D[key][1]) for key in D.keys()])


In [ ]:
import os
from copy import deepcopy

import numpy as np
from PIL import Image, ImageDraw, ImageFont

from shortCardiacBackend.visualisation import concatenate_images, plot_img

# Can be manually adjusted here
show_calculation_step_by_step_folder = r'.'


def init(dicom: np.ndarray, coords: dict):
    # necessary because coords is a dict which works in python as pointer
    return deepcopy(dicom), deepcopy(coords)


def get_img(dicom_img: np.ndarray):
    img_poly = Image.new('RGBA', dicom_img.size)
    img = ImageDraw.Draw(img_poly, 'RGBA')
    dicom_img = dicom_img.convert("RGBA")
    return img, img_poly, dicom_img


def resize(dicom_img: np.ndarray, coords: dict, factor: int):
    for key in coords.keys():
        coords[key] = factor * coords[key]
    dicom_img = dicom_img.resize(tuple((np.array(dicom_img.size[:2]) * factor)))
    return dicom_img, coords


def show_Ref_points_and_septum_axis(dicom_img: np.ndarray, coords: dict):
    dicom_img, coords = init(dicom_img, coords)
    dicom_img, coords = resize(dicom_img, coords, 4)
    img, img_poly, dicom_img = get_img(dicom_img)

    p = tuple([(p1, p2) for p1, p2 in [coords["sacardialRefPoint"][0], coords["sacardialInferiorRefPoint"][0]]])
    img.line(p, fill='red', width=8)
    points = ["P1", "P2"]
    font = ImageFont.truetype("arial.ttf", 150)

    for i in range(2):
        circle_size = 45
        img.ellipse((p[i][0] - circle_size, p[i][1] - circle_size, p[i][0] + circle_size, p[i][1] + circle_size),
                    outline='red', width=8)
        img.text((p[i][0] + 50, p[i][1]), points[i], font=font, fill='red')
    dicom_img.paste(img_poly, mask=img_poly)

    center = ((p[1][0] - p[0][0]) / 2 + p[0][0], (p[1][1] - p[0][1]) / 2 + p[0][1])
    x, y = center[0], center[1]
    h, w = dicom_img.size
    h = round(0.25 * h)
    w = round(0.25 * w)
    dicom_img = dicom_img.crop([x - w, y - h, x + w, y + h])
    dicom_img.save(show_calculation_step_by_step_folder + os.sep + 'Ref_points.png')


def show_coord_correction(dicom_img, coords2, coords, config):
    dicom_img, coords = init(dicom_img, coords)

    p = tuple([(p1, p2) for p1, p2 in [coords["sacardialRefPoint"][0], coords["sacardialInferiorRefPoint"][0]]])

    center = ((p[1][0] - p[0][0]) / 2 + p[0][0], (p[1][1] - p[0][1]) / 2 + p[0][1])
    x, y = center[0], center[1]
    h, w = dicom_img.size
    h = round(0.15 * h)
    w = round(0.15 * w)

    dicom_img = Image.new('RGBA', dicom_img.size, color='white')
    _, coords2 = init(dicom_img, coords2)

    _, coords = resize(dicom_img, coords, 1)
    dicom_img, coords = resize(dicom_img, coords2, 1)

    d = {}
    for ii, coord in enumerate([coords, coords2]):
        coord = [
            coords.get(config.lv_epi_name_or_nr),
            coords.get(config.lv_endo_name_or_nr),
            coords.get(config.rv_name_or_nr),
        ]
        img, img_poly, dicom_img_temp = get_img(dicom_img)
        a = 50
        colors = [(0, 0, 100, a), (0, 100, 0, a), (100, 0, 0, a), (100, 100, 100, a)]

        for i, c in enumerate(coord):
            if c is None:
                continue
            c = tuple([(c1, c2) for c1, c2 in c])
            try:
                color_a = colors[i]
                color_a = (color_a[0], color_a[1], color_a[2], 250)
                img.polygon(c, fill=colors[i], outline=color_a, width=2)
            except:
                pass
        dicom_img_temp.paste(img_poly, mask=img_poly)
        dicom_img = dicom_img_temp.crop([x - w, y - h, x + w, y + h])
        d[str(ii)] = dicom_img_temp
    dicom = concatenate_images(d["0"], d["1"])
    dicom.save(show_calculation_step_by_step_folder + os.sep + 'CordCorrection.png')


def show_angle(dcm_img_pre, coords_pre, dcm_img_post, coords_post, angle):
    dcm_img_pre, coords_pre = init(dcm_img_pre, coords_pre)
    dcm_img_pre, coords_pre = resize(dcm_img_pre, coords_pre, 4)
    img_pre, img_poly_pre, dcm_img_pre = get_img(dcm_img_pre)

    dcm_img_post, coords_post = init(dcm_img_post, coords_post)
    dcm_img_post, coords_post = resize(dcm_img_post, coords_post, 4)
    img_post, img_poly_post, dcm_img_post = get_img(dcm_img_post)

    h, w = dcm_img_pre.size
    h = round(0.25 * h)
    w = round(0.25 * w)

    def plot(img, coords, img_poly, dicom_img, angle):
        p = tuple([(p1, p2) for p1, p2 in [coords["sacardialRefPoint"][0], coords["sacardialInferiorRefPoint"][0]]])
        center = ((p[1][0] - p[0][0]) / 2 + p[0][0], (p[1][1] - p[0][1]) / 2 + p[0][1])
        x = center[0]
        y = center[1]
        circle_size = abs(p[1][1] - p[0][1])
        if angle != 0:
            if angle < 0:
                img.arc((p[1][0] - circle_size, p[1][1] - circle_size, p[1][0] + circle_size, p[1][1] + circle_size),
                        270, 270 - angle, fill=(0, 250, 0, 50), width=circle_size)
            else:
                img.arc((p[1][0] - circle_size, p[1][1] - circle_size, p[1][0] + circle_size, p[1][1] + circle_size),
                        270 - angle, 270, fill=(0, 250, 0, 50), width=circle_size)

        p2 = ((p[1][0], p[1][1] + 300), (p[1][0], p[0][1] - 300))
        img.line(p2, fill='blue', width=8)
        img.line(p, fill='red', width=8)

        dicom_img.paste(img_poly, mask=img_poly)
        dicom_img = dicom_img.crop([x - w, y - h, x + w, y + h])

        return dicom_img

    dcm_img_pre = plot(img_pre, coords_pre, img_poly_pre, dcm_img_pre, angle)
    dcm_img_post = plot(img_post, coords_post, img_poly_post, dcm_img_post, 0)
    dcm = concatenate_images(dcm_img_pre, dcm_img_post)
    dcm.save(show_calculation_step_by_step_folder + os.sep + 'ShowAngle.png')


def show_length_measurements(config, dicom_img, coords, line_params, point_params, angle, crop_img=False, center=None):
    cf_1 = {
        "overlay_dicom": False,
        'overlay_rois': True,
        'overlay_rois_alpha': 0.5,
        "overlay_EI": False,
        'overlay_lines': True,
        'crop_img': crop_img
    }

    img_1 = plot_img(config, cf_1, dicom_img, coords, line_params, point_params, angle,
                     center=center, crop_range=0.15)
    img_1.save(show_calculation_step_by_step_folder + os.sep + 'Length.png')


def show_EIs_measurements(config, dicom_img, coords, line_params, point_params, angle, crop_img=False, center=None):
    cf_1 = {
        "overlay_dicom": False,
        'overlay_rois': True,
        'overlay_rois_alpha': 0.5,
        "overlay_EI": True,
        'overlay_lines': False,
        'crop_img': crop_img}

    img_1 = plot_img(config, cf_1, dicom_img, coords, line_params, point_params, angle,
                     center=center, crop_range=0.15)
    img_1.save(show_calculation_step_by_step_folder + os.sep + 'EIs.png')


def show_center_points(dcm_img, points):
    rv_endo = points[0]
    lv_epi = points[3]
    lv_endo = points[4]
    septum_center = points[5]

    draw, img_poly, dcm_img = get_img(dcm_img)
    color = ['red', 'blue', 'green', 'black']
    font = ImageFont.truetype("arial.ttf", 30)
    names = ['P3', 'P4', 'P5', 'P6']
    for i, p in enumerate([rv_endo, lv_epi, lv_endo, septum_center]):
        circle_size = 5
        draw.ellipse((p[0] - circle_size, p[1] - circle_size, p[0] + circle_size, p[1] + circle_size),
                     outline=color[i], fill=color[i], width=4)
        if i != 2:
            draw.text((p[0], p[1] + 30), names[i], font=font, fill=color[i])
        else:
            draw.text((p[0] + 20, p[1] - 30), names[i], font=font, fill=color[i])

    dcm_img.paste(img_poly, mask=img_poly)

    x, y = septum_center[0], septum_center[1]
    h, w = dcm_img.size
    h = round(0.12 * h)
    w = round(0.12 * w)
    dcm_img = dcm_img.crop([x - w, y - h, x + w, y + h])
    dcm_img.save(show_calculation_step_by_step_folder + os.sep + 'CenterPoints.png')


In [ ]:
import numpy as np
from shortCardiacBackend.transformPointsAndVectors import length, get_vector


def convert_params(line_params: list, point_params: list, area_params: list,
                   scope_params: list, resize_factor: int, pixel_spacing: dict):
    """
    wrapper function to convert als params from pixel spacing to "SI" units

    :param line_params: list with tuples (point1, point2) which define a line
    :param point_params: list with points as numpy array
    :param area_params: list with areas as floats
    :param scope_params: list with scopes as floats
    :param resize_factor: resizing factor between coordinates and dicom pixel
    :param pixel_spacing: dict with keys 'x' and 'y' which load from DICOM pixel spacing

    :return: line_params, point_params, area_paras, scope_params after convertation to "SI" units
    """
    line_params = line_to_si(line_params, resize_factor, pixel_spacing)
    point_params = point_without_resizing(point_params, resize_factor)
    area_params = area_to_si(area_params, resize_factor, pixel_spacing)
    scope_params = scope_to_si(scope_params, resize_factor, pixel_spacing)
    return line_params, point_params, area_params, scope_params


def point_without_resizing(params, resize_factor):
    if params is not None:
        return [(x / resize_factor, y / resize_factor) for x, y in params]


def line_to_si(params: list, resize_factor: int, pixel_spacing: dict):
    """
    convert line params from pixel spacing to "SI" unit - millimeter

    :param params: list with tuples (point1, point2) which define a line
    :param resize_factor: resizing factor between coordinates and dicom pixel
    :param pixel_spacing: dict with keys 'x' and 'y' which load from DICOM pixel spacing

    :return: line_params as list of floats (length in mm)
    """
    if params is not None:
        return [length(get_vector(
            np.array([p[0] / resize_factor * pixel_spacing["x"], p[1] / resize_factor * pixel_spacing["y"]]),
            np.array([q[0] / resize_factor * pixel_spacing["x"], q[1] / resize_factor * pixel_spacing["y"]])
        )) for p, q in params]


def area_to_si(area_params: list, resize_factor: int, pixel_spacing: dict):
    """
    convert area params from pixel spacing to "SI" unit - millimeter ** 2

    :param area_params: list with areas as floats
    :param resize_factor: resizing factor between coordinates and dicom pixel
    :param pixel_spacing: dict with keys 'x' and 'y' which load from DICOM pixel spacing

    :return: list with area_params as list of floats with SI units - mm^2
    """
    if area_params is not None:
        return [area / (resize_factor ** 2) * pixel_spacing["x"] * pixel_spacing["y"] for area in area_params]


def scope_to_si(scope_params, resize_factor, pixel_spacing):
    """

    :param scope_params: list with scopes as floats
    :param resize_factor: resizing factor between coordinates and dicom pixel
    :param pixel_spacing: dict with keys 'x' and 'y' which load from DICOM pixel spacing

    :return:
    """
    if scope_params is not None:
        return [scope / resize_factor * (pixel_spacing["x"] + pixel_spacing["y"]) / 2 for
                scope in scope_params]


In [ ]:
import pydicom


def adjust_gamma(image, gamma=1.0):
    """
    convert image to gamma adjusted image

    :param image: image as np.array
    :param gamma: float

    :return: image after gamma adjustment
    """
    for x in range(image.shape[0]):
        for y in range(image.shape[1]):
            image[x,y] = (image[x,y] / 255)**(1/gamma) * 255
    return image


def get_pixel_spacing(file: str):
    """
    Get pixel spacing of dicom file

    :param: file (str):  path to the file to identify

    :return: spacing (dict): x = x_spacing, y = y_spacing
    """
    x_space = float(pydicom.dcmread(file).PixelSpacing[0])
    y_space = float(pydicom.dcmread(file).PixelSpacing[1])
    return {
        "x": x_space,
        "y": y_space
    }


def flatted_list(lists):
    """
    flat a list of sublist to one large list

    :param lists: list with sublists
    :return: flatted list
    """
    flat_list = []
    for sublist in lists:
        flat_list += sublist
    return flat_list


def is_file_a_dicom(file):
    """
    check if a file is a DICOM file

    :param file: path to the file to identify
    :return: is_dicom - True if the file is DICOM, False otherwise
    """
    try:
        pydicom.dcmread(file)
    except pydicom.errors.InvalidDicomError:
        return False
    return True


def parse_to_arguments(config, coords, dicom_files, uis):
    """
    Arranges the imported coordinates and dicom_files into tuples for the later calculations.
    Note: In DEBUG mode, the number of returned arguments is reduced to 1.

            Parameters:
                    coords (dict): Dictionary with dicom_ui as key, where the extracted coordinates are stored
                    dicom_files (list): List of paths to the Dicom files
                    uis (list): List with all imported Dicom UIs

            Returns:
                    args (tuple): (path to dicom file (str), Coordinates (dict))
    """
    def clear_keys(keys, uis):
        out_keys = []
        for key in keys:
            if key in uis:
                out_keys.append(key)
        return out_keys

    args = [(config, dicom_files[uis.index(ui)], coords.get(ui)) for ui in clear_keys(coords.keys(), uis)]
    if config.DEBUG:
        print("WARNING: DEBUG_MODE ONLY REDUCED LIST OF ARGUMENTS FOR THE CALCULATIONS")
        args = args[:1]
    return args


In [ ]:
import itertools

import SimpleITK as sitk
import numpy as np
import pydicom
from PIL import Image, ImageDraw
from imantics import Mask
from numba import jit
from scipy.interpolate import splprep, splev
from skimage.transform import resize

from shortCardiacBackend.transformPointsAndVectors import find_smallest_distance_to_ref, clean_points, get_line, \
    rotate_points, length, get_vector, calc_center_of_2_points


def order_roi_to_dict(roi: list, delta: int = 1):
    """
    Sorting of points concerning x-coordinate

    :param roi: list of points as np.array
    :param delta: delta of x-values allowed to fill gaps and stabilize calculations concerning single contour points

    :return: point_dict_out (dict): Dictionary in which points are stored that have the same x-coordinate
    """
    point_dict = {}
    for point in roi:
        x_key = str(int(float(point[0])))
        if x_key not in point_dict.keys():
            point_dict[x_key] = [point]
        else:
            point_dict[x_key].append(point)
    point_dict_out = {}
    for key in point_dict.keys():
        points_list = []
        for i in range(-delta, delta + 1):
            points = point_dict.get(str(int(float(key)) + i))
            if points is not None:
                points_list += points
        point_dict_out[key] = clean_points(points_list, key)
    return point_dict_out


def smooth_coord_resizing(coords, resize_polygon_factor, smoothing=False, mode='int'):
    """
    Resizing and smoothing (using an approximating spline curve) the coordinates of a parameter.
    Notes: The function is implemented only for 2D calculations

            Parameters:
                    coords (np.array): Numpy array with the coordinates for a parameter (shape - (number of points,2))
                    resize_polygon_factor (int): Factor by which the coordinates are scaled up for a more accurate calculation
                    smoothing (bool): Specifies whether smoothing should be performed or only resizing should be performed.
                    Mode (str): Specifies whether only pixel coordinates (integers) or subpixel coordinates (float) should be used for the calculation.

            Returns:
                    coords (dict): Dictionary with all imported coordinates after resizing.
    """
    # If it is a parameter with only one point, no smoothing is performed, and the point is rescaled.
    coords = coords * resize_polygon_factor
    if coords.shape[0] != 1 and smoothing:
        x = coords[:, 0]
        y = coords[:, 1]
        d1 = (x.max() - x.min())
        d2 = (y.max() - y.min())
        # Approximation of the necessary pixels for an exact calculation of the parameters using the circumference of an ellipse
        number_of_points = (np.pi * (3 * (d1 + d2) / 2 - np.sqrt(d1 * d2))) / resize_polygon_factor
        tck, _ = splprep([x, y], s=0, per=True)
        xx, yy = splev(np.linspace(0, 1, round(number_of_points)), tck, der=0)
        coords = np.array([xx, yy]).transpose((1, 0))
    if mode == 'int':
        return coords.round().astype('int16')
    return coords


def coords_resizing(coords, resize_polygon_factor, smooth_resizing: bool = True, mode='int'):
    """
    Wrapper function calls the smooth_coord_resizing() function for all parameters found in coords.

            Parameters:
                    coords (dict): Dictionary in which all imported coordinates are stored.
                    resize_polygon_factor (int): Factor by which the coordinates are scaled up for a more accurate calculation
                    smooth_resizing (bool):
                    mode (str): Specifies whether only pixel coordinates (integers) or subpixel coordinates should be used for the calculation (float).

            change:
                    coords (dict): Dictionary with all imported coordinates after resizing
    """
    for key in coords.keys():
        coords[key] = smooth_coord_resizing(np.array(coords[key]), resize_polygon_factor, smooth_resizing, mode)


def find_ref_points(coords, config):
    right_ventricel = list(coords.get(config.rv_name_or_nr))
    left_ventricle = list(coords.get(config.lv_epi_name_or_nr))
    r_points, l_points = [], []
    min_delta = config.resize_polygon_factor
    count = 0
    while len(r_points) < 10:
        count += 1
        if count == 4:
            coords["sacardialRefPoint"] = None
            return None
        for p, q in itertools.product(right_ventricel, left_ventricle):
            l = length(get_vector(p, q))
            if l < min_delta:
                r_points.append(p)
                l_points.append(q)
        min_delta *= 2

    def calc_refpoints(points):
        distance, ref_points = [], []
        for p, q in itertools.product(points, points):
            distance.append(length(get_vector(p, q)))
            ref_points.append((p, q))
        ref = ref_points[np.nanargmax(distance)]
        if ref[0][1] > ref[1][1]:
            ref = (ref[1], ref[0])
        return ref

    l_ref = calc_refpoints(l_points)
    r_ref = calc_refpoints(r_points)
    coords["sacardialRefPoint"] = np.array([calc_center_of_2_points(np.array(l_ref[0]), np.array(r_ref[0]))])
    coords["sacardialInferiorRefPoint"] = np.array([calc_center_of_2_points(np.array(l_ref[1]), np.array(r_ref[1]))])


def correct_segmentations(coords, config):
    right_ventricel = list(coords.get(config.rv_name_or_nr))
    left_ventricle = list(coords.get(config.lv_epi_name_or_nr))

    # split left
    p1_l = find_smallest_distance_to_ref(left_ventricle, coords["sacardialRefPoint"])
    p2_l = find_smallest_distance_to_ref(left_ventricle, coords["sacardialInferiorRefPoint"])
    roi_1_l, roi_2_l = split_polygon(left_ventricle, p1_l, p2_l)

    # split right
    p1_r = find_smallest_distance_to_ref(right_ventricel, coords["sacardialRefPoint"])
    p2_r = find_smallest_distance_to_ref(right_ventricel, coords["sacardialInferiorRefPoint"])
    roi_1_r, roi_2_r = split_polygon(right_ventricel, p1_r, p2_r)

    def reverse_list(l):
        l.reverse()
        return l

    left_ventricle = roi_1_l + reverse_list(roi_2_r)

    coords[config.lv_epi_name_or_nr] = np.array(left_ventricle)

    coords["sarvendocardialContourVentral"] = np.array(roi_1_r)
    coords["sarvendocardialContourDorsal"] = np.array(roi_2_r)
    coords["sarvendocardialContourDorsalClosed"] = np.array(
        roi_2_r + list(get_line(coords["sacardialInferiorRefPoint"][0] / (0.5 * config.resize_polygon_factor),
                                coords["sacardialRefPoint"][0] / (0.5 * config.resize_polygon_factor), numpy=True) * 0.5 * config.resize_polygon_factor))


def split_polygon(points, p1, p2):
    """

    :param points:
    :param p1:
    :param p2:
    :return:
    """
    roi_1, roi_2, no_contour, current_contour = [], [], [], 'no_contour'
    roi_1_no_contour, roi_2_no_contour = [], []

    for p in points:
        if all(p == p1) and len(roi_2) == 0:
            if len(no_contour) > 0:
                no_contour.append(p)
            roi_1_no_contour = no_contour
            no_contour = []
            current_contour = 'roi_2'
            roi_2.append(p)
            continue

        if all(p == p2) and len(roi_1) == 0:
            if len(no_contour) > 0:
                no_contour.append(p)
            roi_2_no_contour = no_contour
            no_contour = []
            current_contour = 'roi_1'
            roi_1.append(p)
            continue
        if current_contour == 'roi_1':
            roi_1.append(p)
        if current_contour == 'roi_2':
            roi_2.append(p)
        if current_contour == 'no_contour':
            no_contour.append(p)

    roi_2 = roi_2 + roi_2_no_contour
    roi_1 = roi_1 + roi_1_no_contour

    l2 = calc_polygon_length(roi_2)
    l1 = calc_polygon_length(roi_1)
    if l1 > l2:
        return roi_1, roi_2
    return roi_2, roi_1


def calc_line_for_EI(roi, angle):
    """
    Calculation of the longest diameter along an angle for the analysis of the EI.
    Note: For more information, see Yamasaki et al - doi: ?????


            Parameters:
                    ROI (np.array): array in which the coordinates of a contour are stored - dimensions (numer_of_points, 2).
                    angle (float): angle to the heart axis along which the longest diameter is to be determined

            returns:
                    points (list): list of the two points defining the longest diameter [p1 (np.array), p2 (np.array)]
    """
    center = calc_center_of_polygon(roi)
    roi = rotate_points(roi, center, angle, 'int')
    point_dict = order_roi_to_dict(roi)
    xs = [int(float(key)) for key in point_dict.keys()]
    max_dist = 0
    line = None
    range_ = round((max(xs) - min(xs)) / 15)
    for i in range(min(xs) + range_, max(xs) - range_ + 1):
        dists = [point_dict[str(k)][2] if point_dict.get(str(k)) is not None else None for k in
                 range(i - range_, i + range_)]
        for _ in range(dists.count(None)):
            dists.remove(None)
        if len(dists) > 0:
            dist = np.array(dists).mean()

            if dist > max_dist:
                line = point_dict[str(i)][:2] if point_dict.get(str(i)) is not None else line
                max_dist = dist if point_dict.get(str(i)) is not None else max_dist
    return rotate_points(line, center, -angle)


@jit(nopython=True)
def calc_center_of_polygon(points):
    """
    Determines the centroid of a polygon

    :param points: 2D numpy array with points [(p1_x, p1_y),(p2_x, p2_y),...]

    :return: Array with the coordinates of the center point (x, y)
    """
    A = calc_area(list(points), False)
    x = points[:, 0]
    y = points[:, 1]
    n = len(points)
    xs = 0.0
    ys = 0.0
    for i in range(n):
        j = (i + 1) % n
        xs += (x[i] + x[j]) * (x[i] * y[j] - x[j] * y[i])
        ys += (y[i] + y[j]) * (x[i] * y[j] - x[j] * y[i])
    return np.array([round(xs / (6 * A)), round(ys / (6 * A))])


@jit(nopython=True)
def calc_area(corners, absolute=True):
    """
    Determines the area of a polygon

    :param corners: np.array of length n with the points of the respective ROI [(x1,y1),(x2,y2),...,(xn,yn)]
    :param absolute: Specifies whether there may also be negative surfaces

    :return: area (float): Area of contour
    """

    area = 0.0
    n = len(corners)
    for i in range(n):
        j = (i + 1) % n
        area += corners[i][0] * corners[j][1]
        area -= corners[j][0] * corners[i][1]
    area = area / 2.0
    if absolute:
        return abs(area)
    return area


def calc_polygon_length(points):
    """
    Determines the scope of a polygon

    :param points:  np.array of length n with the points of the respective ROI [(x1,y1),(x2,y2),...,(xn,yn)]

    :return: scope of a polygon as float
    """
    polygon_length = 0
    for i in range(len(points) - 1):
        polygon_length += length(get_vector(points[i], points[i + 1]))
    polygon_length += length(get_vector(points[0], points[-1]))
    return polygon_length


def smooth_3D_mask(mask, scaling=2):
    mask = resize(mask, (mask.shape[1] * scaling, mask.shape[0] * scaling), order=0)
    sitk_mask = sitk.GetImageFromArray(mask.astype('int8'))
    return sitk.GetArrayFromImage(sitk.BinaryDilate(sitk.BinaryMorphologicalClosing(sitk.BinaryErode(sitk_mask))))


def mask_to_polygon(mask):
    """
    calc polygon of binary mask

    :param mask: np.array as binary mask

    :return: polygon as numpy array
    """
    polygon = list(Mask(mask).polygons().points[0])

    def line_interp(points):
        inter_p = []
        for i in range(len(points)):
            j = 0 if i == len(points) - 1 else i + 1
            inter_p.append(points[i])
            inter_p.append(calc_center_of_2_points(points[i], points[j], mode='float'))
        return inter_p

    polygon = line_interp(line_interp(polygon))
    return np.array(polygon)


def calc_mask_of_polygon(dcm_file, coord, name, scaling_factor):
    """
    calculation of binary mask from polygon

    :param dcm_file: path to dicom file
    :param coord: dict with coords
    :param name: name / key for polygon in coord dict
    :param scaling_factor: int

    :return: binary numpy array
    """
    dcm_shape = pydicom.dcmread(dcm_file).pixel_array.shape
    img = Image.new('L', (dcm_shape[1] * scaling_factor, dcm_shape[0] * scaling_factor), 0)
    img_draw = ImageDraw.Draw(img)
    c = tuple([(c1 * scaling_factor, c2 * scaling_factor) for c1, c2 in list(coord.get(name))])
    img_draw.polygon(c, outline=1, fill=1)
    return np.array(img)


In [ ]:
def line_intersection(line1, line2):
    """
    Calculates intersection of two lines

        Parameters:
            line1 (list): list with two points [point_1, point_2]
            line2 (list):  list with two points [point_1, point_2]

        Returns:
            intersection (tuple):(x, y)
    """
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
        div = 1
    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return x, y


@jit(nopython=True)
def get_vector(p0, p1):
    """
    Calculates the vector between two points, with len n = Number of dimensions

            Parameters:
                    p0 (np.array): np-array with shape (n,) - [d_1, d_2, d_3, ..., d_n]
                    p1 (np.array): np-array with shape (n,) - [d'_1, d'_2, d'_3, ..., d'_n]

            Returns:
                    p (np.array): np-array with shape (n,) - [d_1 - d'_1, d_2 - d'_2, d_3 - d'_3, ..., d_n - d'_n]
    """

    return p0 - p1


@jit(nopython=True)
def dotproduct(v1, v2):
    """
    Calculates dotproduct of two vectors

            Parameters:
                    v1 (np.array): vector as np-array with shape (n,) - [d_1, d_2, d_3, ..., d_n]
                    v2 (np.array): vector as np-array with shape (n,) - [d'_1, d'_2, d'_3, ..., d'_n]

            Returns:
                    dotproduct (float): v1 \dot v2
    """
    return (v1 * v2).sum()


@jit(nopython=True)
def length(v):
    """
    Calculates the length of vector v

            Parameters:
                    v (np.array): vect as np-array with shape (n,) - [d_1, d_2, d_3, ..., d_n]

            Returns:
                    length (float): ength of vector
    """
    scale = np.abs(v).sum()
    if scale == 0:
        return 0
    return np.sqrt(dotproduct(v/scale, v/scale))*scale


@jit(nopython=True)
def calc_angle(v1, v2):
    """
    Calculates the angle between two vectors in degree

            Parameters:
                    v1 (np.array): vector as np-array with shape (n,) - [d_1, d_2, d_3, ..., d_n]
                    v2 (np.array): vector as np-array with shape (n,) - [d'_1, d'_2, d'_3, ..., d'_n]

            Returns:
                    angle (float): angle in degree
    """
    length_v1 = length(v1)
    length_v2 = length(v2)
    if length_v1 == 0.0 or length_v2 == 0.0:
        return 1000
    value = dotproduct(v1/length_v1, v2/length_v2)
    return np.round(np.rad2deg(np.arccos(value)) * 1000) / 1000


def calc_center_of_2_points(p1, p2, mode='int'):
    """
    Returns center point between 2 points

            Parameters:
                    p1 (np.array): Point 1
                    p2 (np.array): Point 2
                    mode (str): Specifies if the midpoint should be exactly on one pixel or if subpixels are also possible

            returns:
                    center (np.array): Center point between p1 and p2
    """
    center = p1 + (p2 - p1) / 2
    if mode == 'int':
        return center.round().astype('int16')
    return center


def get_line(p1, p2, numpy=False):
    """
    Returns line with each point between 2 points

            Parameters:
                    p1 (np.array): Point 1
                    p2 (np.array): point 2
                    numpy (bool): Specifies whether line (list of points) should be determined as numpy array

            Returns:
                    line (list \ np. array): List with points between the two points
    """
    x1, y1 = p1.astype(np.int16)
    x2, y2 = p2.astype(np.int16)
    points = []
    issteep = abs(y2 - y1) > abs(x2 - x1)
    if issteep:
        x1, y1 = y1, x1
        x2, y2 = y2, x2
    rev = False
    if x1 > x2:
        x1, x2 = x2, x1
        y1, y2 = y2, y1
        rev = True
    deltax = x2 - x1
    deltay = abs(y2 - y1)
    error = round(deltax / 2)
    y = y1
    ystep = 1 if y1 < y2 else -1
    for x in range(x1, x2 + 1):
        if issteep:
            points.append((y, x))
        else:
            points.append((x, y))
        error -= deltay
        if error < 0:
            y += ystep
            error += deltax
    # Reverse the list if the coordinates were reversed
    if rev:
        points.reverse()
    if numpy:
        return np.array(points)
    return points


def rotate_points(points, origin, angle, mode='int'):
    """
    2D point transformation from a set of points around a clockwise origin.

            Parameters:
                    points (list): list of points (each point is an np.array)
                    origin (np.array): point around which the other points are rotated
                    angle (float): angle in degrees around which the points are rotated
                    mode (str): int or float - Specifies if points should be rotated to whole pixels only or if there should be subpixels

            Returns:
                    dcm_img (Pillow Image): dcm_image as RGB image
    """
    new_point_params = []
    if points is None:
        return new_point_params
    for point in points:
        if point is None:
            continue
        new_point_params.append(np.array(rotate_point(origin, point, angle, mode)))
    return new_point_params


@jit(nopython=True)
def rotate_point(origin, point, angle, mode='int'):
    """
    Rotate a point counterclockwise by a given angle around a given origin.

            Parameters:
                origin (np.array) = origin
                point (np.array) = point for rotation
                angle (float) = angle in degree
                mode (str) = Specification whether the point should be rotated to a whole pixel only or whether there are subpixels

            Returns:
                point (np.array): point after rotation
    """
    angle = angle * math.pi / 180
    ox, oy = origin
    px, py = point

    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    if mode == 'int':
        return round(qx), round(qy)
    return qx, qy


def find_smallest_distance_to_ref(points, ref):
    """
    Returns the point from a set of points with the smallest distance to a reference point.

            Parameters:
                    points (list): List with points (points are available as np.array)
                    ref (np.array): Coordinates of the reference point

            Returns:
                    point (np.array): Nearest point
    """
    dist = [length(get_vector(np.array(p), ref)) for p in points]
    return points[np.argmin(dist)]


def find_distance_with_angles_and_fixed_point(points: list, angles: list, fixed_point: np.array):
    """
    Determines the maximum distances from a reference point to a contour (list of points) at different angles

            Parameters:
                    points (list): contour \ polygon as a list with points as np.array
                    angles (list): list with angles to be examined in degrees
                    fixed_point (np.array): coordinates of the reference point (x,y)

            returns:
                    distances (dict): Dictionary with the angle as key and the distances as a tuple with two coordinates
                    (point on contour, reference point)
    """
    D = {str(angle): None for angle in angles}
    for angle in angles:
        points_rot = [rotate_point(fixed_point, point, -angle) for point in points]
        if angle >= 0:
            points_rot = [point_rot if point_rot[1] > fixed_point[1] else np.array([-100, -100]) for point_rot in
                          points_rot]
        else:
            points_rot = [point_rot if point_rot[1] < fixed_point[1] else np.array([-100, -100]) for point_rot in
                          points_rot]
        delta_x = [abs(p[0] - fixed_point[0]) for p in points_rot]
        D[str(angle)] = [np.array(points[np.argmin(delta_x)]), np.array(fixed_point)]
    return D


def clean_points(points, key, mode='int'):
    """
     If there are multiple points with the same x-coordinate, all points that do not have the longest distance are removed.

            Parameters:
                    points (list): contour \ polygon as a list with points as np.array
                    key (str): x-coordinate
                    mode (str): coordinates of the reference point (x,y)

            Returns:
                    p1 (np.array): coordinates point 1
                    p2 (np.array): Coordinates point 2
                    length (float): Length between the two points
    """
    count = 0
    for p in points:
        if p is None:
            count += 1
    for _ in range(count):
        points.remove(None)
    if len(points) < 2:
        return None
    idx_min = np.argmax(np.array(points)[:, 1])
    idx_max = np.argmin(np.array(points)[:, 1])
    p1 = np.array(points[idx_min])
    p2 = np.array(points[idx_max])
    p1[0] = float(key)
    p2[0] = float(key)
    if mode == 'int':
        p1 = p1.round().astype('int16')
        p2 = p2.round().astype('int16')
    return p1, p2, length(get_vector(p1, p2))

In [ ]:
import numpy as np
from PIL import Image, ImageDraw

from shortCardiacBackend.transformContours import calc_center_of_polygon
from PIL import Image, ImageDraw, ImageFont
from shortCardiacBackend.transformContours import rotate_points


def show_segmentation(config, dicom_img, coords):
    """

    :param dicom_img: dicom as PIL image
    :param coords: list of np-array with coords
    :param line_params: list with np-array of two points that define a line [[(px_1,py_1),(px_2,py_2)],...]
    :param point_params: list with points [np.array([px_1,py_1]), np.array([px_2,py_2]), ....]
    :param resize: Resize factor to save the image with a higher resolution.
    :return: resized dicom as PIL image
    """
    # plot_all_contours(coords, dicom_img, angle)

    coords = [
        coords.get(config.lv_epi_name_or_nr),
        coords.get(config.lv_endo_name_or_nr),
        coords.get(config.rv_name_or_nr),
    ]

    dicom_img = dicom_img.convert("RGBA")
    if config.img_transparent:
        back = Image.new('RGBA', dicom_img.size)
    else:
        back = Image.new('RGBA', dicom_img.size, color='white')

    img_poly = Image.new('RGBA', dicom_img.size)
    img = ImageDraw.Draw(img_poly, 'RGBA')
    a = 150
    colors = [(0, 0, 100, a), (0, 100, 0, a), (100, 0, 0, a), (100, 100, 100, a)]

    for i, c in enumerate(coords):
        if c is None:
            continue

        c = tuple([(c1, c2) for c1, c2 in c])
        try:
            img.polygon(c, fill=colors[i], outline=colors[i])
        except:
            pass
    dicom_cut_out = dicom_img.load()
    for x in range(img_poly.size[0]):
        for y in range(img_poly.size[1]):
            dicom_cut_out[x, y] = (dicom_cut_out[x, y][0], dicom_cut_out[x, y][1], dicom_cut_out[x, y][2], 0)

    # dicom_img.paste(img_poly, mask=img_poly)
    back.paste(img_poly, mask=img_poly)
    if config.second_img:
        dicom_img = concatenate_images(dicom_img, back)
    return dicom_img


def rotate_img(img, angle):
    return img.rotate(angle)


def concatenate_images(img1, img2):
    """
    Merges two images side by side

            Parameters:
                    img1 (Pillow Image): Image 1 (left)
                    img2 (Pillow Image): Image 2 (right)

            Returns:
                    img_conc (Pillow Image): Image 1 and image 2 next to each other
    """
    # creating a new image
    img_conc = Image.new("RGBA",
                         (img1.size[0] + img2.size[0], img1.size[1] if img1.size[1] > img2.size[1] else img2.size[1]))

    # pasting the first image (image_name, (position))
    img_conc.paste(img1, (0, 0))

    # pasting the second image (image_name, (position))
    img_conc.paste(img2, (img1.size[0], 0))
    return img_conc


def visualization(config, dicom_img, coords, line_params, point_params, angle, crop_img=False, center=None):
    cf_1 = {
        "overlay_dicom": config.first_img_overlay_dicom,
        'overlay_rois': config.first_img_overlay_rois,
        'overlay_rois_alpha': config.first_img_overlay_rois_alpha,
        "overlay_EI": config.first_img_overlay_EI,
        'overlay_lines': config.first_img_overlay_lines,
        "img_transparent": config.img_transparent,
        'crop_img': 'show_box',  # crop_img,
        'angle_correction': False
    }

    cf_2 = {
        'overlay_dicom': config.second_img_overlay_dicom,
        'overlay_rois': config.second_img_overlay_rois,
        'overlay_rois_alpha': config.second_img_overlay_rois_alpha,
        'overlay_EI': config.second_img_overlay_EI,
        'overlay_lines': config.second_img_overlay_lines,
        "img_transparent": False,
        'crop_img': crop_img,
        'angle_correction': True
    }

    img_1 = plot_img(config, cf_1, dicom_img, coords, line_params, point_params, angle,
                     center=center, crop_range=0.15)
    if config.second_img:
        img_2 = plot_img(config, cf_2, dicom_img, coords, line_params, point_params, angle,
                         center=center, crop_range=0.15, scaling=3)

        if img_2.size != img_1.size:
            img_2 = img_2.resize(img_1.size)
        img_1 = concatenate_images(img_1, img_2)
        # img_1 = show_legend(img_1)
        b = 2
    return img_1


def show_legend(img):
    draw = ImageDraw.Draw(img, 'RGBA')
    x = img.size[0] / 2 + 20
    y = img.size[1] - img.size[1] / 3
    delta_y = 80
    colors = ['red'] + ['orange'] + ['blue'] + ['green'] + ['black']
    name = ['Septum Center to ventral right ventricle endocard contour',
            'Septum Center to dorsal right ventricle endocard contour',
            'Center to left ventricle epicard contour',
            'Center to left ventricle endocard contour',
            'angle-dependent length measurements']
    font = ImageFont.truetype("arial.ttf", 60)
    for i in range(5):
        if i < 4:
            p = tuple([(x, y - delta_y * i), (x + 20, y - delta_y * i)])
            draw.line(p, fill=colors[i], width=6)
        if i == 4:
            font = ImageFont.truetype("arial.ttf", 80)
        draw.text((x + 40, y - delta_y * i - 20), name[i], font=font, fill=colors[i])
    return img


def plot_img(config, img_cf, dicom_img, coords, line_params, point_params, angle, center=None, crop_range=None,
             scaling=None):
    """

    :param dicom_img: dicom as PIL image
    :param coords: list of np-array with coords
    :param line_params: list with np-array of two points that define a line [[(px_1,py_1),(px_2,py_2)],...]
    :param point_params: list with points [np.array([px_1,py_1]), np.array([px_2,py_2]), ....]
    :param resize: Resize factor to save the image with a higher resolution.
    :return: resized dicom as PIL image
    """
    if scaling is None or img_cf['crop_img'] == False:
        scaling = 1
    center = center * scaling
    pre_size = dicom_img.size
    dicom_img = dicom_img.resize((scaling * pre_size[0], scaling * pre_size[1]))
    center_img = (dicom_img.size[0] / 2, dicom_img.size[1] / 2)
    if not img_cf["overlay_dicom"]:
        if img_cf["img_transparent"]:
            dicom_img = Image.new('RGBA', dicom_img.size)
        else:
            dicom_img = Image.new('RGBA', dicom_img.size, color='white')
    if img_cf["angle_correction"]:
        dicom_img = dicom_img.convert("RGBA").rotate(-angle)
    else:
        dicom_img = dicom_img.convert("RGBA")
    img_poly = Image.new('RGBA', dicom_img.size)
    img = ImageDraw.Draw(img_poly, 'RGBA')
    coords = [
        coords.get(config.lv_epi_name_or_nr) * scaling,
        coords.get(config.lv_endo_name_or_nr) * scaling,
        coords.get(config.rv_name_or_nr) * scaling,
    ]
    # Wenn in der Config Datei angegeben, werden die eingezeichneten ROIs überlagert
    if img_cf["overlay_rois"]:
        a = int(255 * img_cf["overlay_rois_alpha"])

        colors = [(0, 0, 100, a), (0, 100, 0, a), (100, 0, 0, a), (100, 100, 100, a)]

        for i, c in enumerate(coords):
            if c is None:
                continue
            c = tuple([(c1, c2) for c1, c2 in c])
            if not img_cf['angle_correction']:
                c = rotate_points(c, center_img, -angle)
                c = [tuple(_) for _ in c]
            try:
                color_a = colors[i]
                color_a = (color_a[0], color_a[1], color_a[2], 250)
                img.polygon(c, fill=colors[i], outline=color_a)
            except:
                pass
    # TODO: Remove after Paper exception only necessary for 3D plot
    """
    if True:
        img_cut_off = Image.new('RGBA', dicom_img.size)
        draw = ImageDraw.Draw(img_cut_off, 'RGBA')
        for i, c in enumerate(coords):
            if c is None:
                continue
            c = tuple([(c1, c2) for c1, c2 in c])
            try:
                draw.polygon(c, fill='red', outline='red')
            except:
                pass
        cut_out = img_cut_off.load()
        dicom_cut_out = dicom_img.load()
        for x in range(img_poly.size[0]):
            for y in range(img_poly.size[1]):
                if cut_out[x,y][0] > 0 or cut_out[x,y][1] > 0 or cut_out[x,y][2] > 0:
                    continue
                dicom_cut_out[x,y] = (dicom_cut_out[x,y][0], dicom_cut_out[x,y][1], dicom_cut_out[x,y][2], 50)
    """
    colors = ['red'] + ['orange'] + ['blue'] + ['green'] + ['red'] + ['blue'] + ['green'] + ['orange'] + ['black'] * 100
    width = [2 * scaling, 2 * scaling, 2 * scaling, 2 * scaling, 6 * scaling, 6 * scaling, 6 * scaling, 6 * scaling]
    for i, lines in enumerate(line_params):
        lines = [(p1 * scaling, p2 * scaling) for p1, p2 in lines]
        if not img_cf['angle_correction']:
            lines = [tuple(rotate_points(line, center_img, -angle)) for line in lines]
        if lines is None:
            continue
        if i < 4 and not img_cf["overlay_lines"]:
            continue
        if i >= 4 and not img_cf["overlay_EI"]:
            continue
        for p in lines:
            if p is None:
                continue
            p = tuple([(p1, p2) for p1, p2 in p])
            img.line(p, fill=colors[i], width=width[i])
            if p[0][0] == p[1][0] and i >= 4:
                min_, max_ = p[0][0] - 10, p[0][0] + 10
                img.line(((min_, p[0][1]), (max_, p[0][1])), fill=colors[i], width=width[i])
                img.line(((min_, p[1][1]), (max_, p[1][1])), fill=colors[i], width=width[i])
            if p[0][1] == p[1][1] and i >= 4:
                min_, max_ = p[0][1] - 10, p[0][1] + 10
                img.line(((p[0][0], min_), (p[0][0], max_,)), fill=colors[i], width=width[i])
                img.line(((p[1][0], min_), (p[1][0], max_,)), fill=colors[i], width=width[i])
    if img_cf['crop_img'] == 'show_box':
        x = center[0]
        y = center[1]
        h, w = dicom_img.size
        p1 = (x - round(0.25 * h), y - round(0.25 * h)) if crop_range is None else (
        x - round(crop_range * h), y - round(crop_range * h))
        p2 = (x - round(0.25 * h), y + round(0.25 * h)) if crop_range is None else (
        x - round(crop_range * h), y + round(crop_range * h))
        p3 = (x + round(0.25 * h), y + round(0.25 * h)) if crop_range is None else (
        x + round(crop_range * h), y + round(crop_range * h))
        p4 = (x + round(0.25 * h), y - round(0.25 * h)) if crop_range is None else (
        x + round(crop_range * h), y - round(crop_range * h))
        points = [p1, p2, p3, p4, p1]
        if not img_cf['angle_correction']:
            points = rotate_points([p1, p2, p3, p4, p1], center_img, -angle)
        img.polygon([tuple(p) for p in points], outline='red', width=6)

    dicom_img.paste(img_poly, mask=img_poly)

    if type(img_cf['crop_img']) == bool:
        if img_cf['crop_img']:
            x = center[0]
            y = center[1]
            h, w = dicom_img.size
            h = round(0.25 * h) if crop_range is None else round(crop_range * h)
            w = round(0.25 * w) if crop_range is None else round(crop_range * w)
            dicom_img = dicom_img.crop([x - w, y - h, x + w, y + h])

    return dicom_img.convert("RGBA")


In [ ]:
def run(arg):
    if arg[2] is None:
        return None
    sCardiac = ShortCardiac(config=arg[0], dcm_file=arg[1], coords=arg[2])
    results = sCardiac.run()
    del sCardiac
    return results


def main(coord_file, dcm_folder, config):
    freeze_support()

    # Reading in the Dicom images
    dicoms, UIs = load_DICOMs(dcm_folder)
    # Transformation of the coordinates into a file format optimized for shortCardiac Notes: Conversion can take
    # longer depending on the size, but conversion is only necessary for the first calculation. If the data has
    # already been converted, this step can be skipped.
    coordReader = CoordReader(config)
    if not coord_file.endswith('.pkl'):
        if config.coord_mode == 'cvi42':
            cvi42_file = coord_file
            coord_file = coord_file[:-4] + '.pkl'
            coordReader.preparation_cvi42(file=cvi42_file, UIs=UIs, save_file_name=coord_file)
        elif config.coord_mode == 'nii':
            nii_file = coord_file
            coord_file = coord_file[:-4] + '.pkl' if 'nii.gz' not in coord_file else coord_file[:-7] + '.pkl'
            coordReader.preparation_nii(nii_file=nii_file, dcm_sorted=dicoms, save_file_name=coord_file)
        else:
            raise UserWarning

    # Reading in the coordinates
    coords = coordReader.load_coordinates(coord_file)

    args = parse_to_arguments(config, coords, dicoms, UIs)

    if config.worker == 0:
        results = [run(arg) for arg in args]
    else:
        with Pool(config.worker) as pool:
            results = [_ for _ in tqdm(pool.imap_unordered(run, args), total=len(args))]

    results = natsort.natsorted(results)
    save(results, dcm_folder)


In [ ]:
# With this script you can test the function of shortCardiac.
# The calculated parameters can be found in TestData/DICOM/*
nii_file = r'TestData/mask.nii.gz'
dcm_folder = r'TestData/Dicom'
config = RunConfiguration()

config.rv_name_or_nr = '1'
config.lv_epi_name_or_nr = '2'
config.lv_endo_name_or_nr = '3'
config.worker = 0

main(nii_file, dcm_folder, config)